In [75]:
import numpy as np
import pandas as pd
import re

In [76]:
from nltk.stem import SnowballStemmer
stemmer_eng = SnowballStemmer('english')
stemmer_rus = SnowballStemmer('russian')

In [77]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [78]:
df_test.head(5)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


# Preprocessing

In [79]:
black_list = []
for sub_df in df_train.groupby('url'):
    
    if ( sub_df[1].target.count() >1 ):
        if ( all(sub_df[1].target) ):
            
            black_list.append(sub_df[0])

In [80]:
df_test['dirt_url'] = df_test.url.map(lambda x: 1 if x in black_list else 0)
df_train['dirt_url'] = df_train.url.map(lambda x: 1 if x in black_list else 0)

In [81]:
file = open("stopwords.txt", "r")
stopwords_eng_rus = [line[:-1] for line in file]

In [82]:
def Clean(string):
    string = string.lower()
    string = re.sub(r"\s+" ," ", re.sub(r"[^a-zа-яё]", " ", string))

    string = string.split(' ')
    tokens = []
    for word in string:
        if (word in stopwords_eng_rus) or word=='':
            continue
        else: 
            tokens.append(word)
            
    for i in range (len(tokens)):
        tokens[i] = stemmer_eng.stem(tokens[i])
        tokens[i] = stemmer_rus.stem(tokens[i])
        
    return tokens

In [ ]:

df_train['title_tokens'] = df_train['title'].map(lambda x: Clean(x))
df_test['title_tokens'] = df_test['title'].map(lambda x: Clean(x))

In [ ]:
df_train.head(10)

In [ ]:
df_test.head(100)

In [ ]:
df_train['title_tokens']=df_train['title_tokens'].map(lambda x:' '.join(x))
df_test['title_tokens']=df_test['title_tokens'].map(lambda x:' '.join(x))

In [ ]:
def Clean2(string):
    if len(string)<3:
        if len(re.sub(r"\s", "", string))<3: 
            return ' '
    return string

df_train['clean_title'] = df_train['title_tokens'].map(lambda x: Clean2(x))
df_test['clean_title'] = df_test['title_tokens'].map(lambda x: Clean2(x))

In [ ]:
#df['clean_title'] = df['title_tokens'].map(lambda x: x.split(' '))

# Naive Baise

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
from sklearn.naive_bayes import MultinomialNB
MNB_model = MultinomialNB(alpha=0.09)

In [23]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [24]:
tfidf_vectorizer.fit(df_train['clean_title'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [25]:
len(tfidf_vectorizer.get_feature_names())

627624

In [26]:
vect_title = tfidf_vectorizer.transform(df_train['clean_title'])

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
y_train = df_train["target"].astype(int).values

In [29]:
from sklearn.metrics import f1_score

In [30]:
MNB_model.fit(vect_title, y_train)

MultinomialNB(alpha=0.09, class_prior=None, fit_prior=True)

In [31]:
f1_score(y_train, MNB_model.predict(vect_title))

0.9843314658917284

In [32]:
df_train['NB_title'] = MNB_model.predict(vect_title)

# LogRegression


In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
LogReg_model = LogisticRegression(C=100, n_jobs=-1)

In [35]:
LogReg_model.fit(vect_title, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1297: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
f1_score(y_train, LogReg_model.predict(vect_title))

0.99249796631617

In [37]:
df_train['LogReg_title'] = LogReg_model.predict(vect_title)

# TreeClassifier

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier(criterion='entropy', random_state=7)

In [61]:
tree_params = {'max_depth': range(1, 15, 3),
               'max_features': range(1, 5),
                'min_samples_leaf': [50, 100, 200]}

In [62]:
tree_grid = GridSearchCV( clf_tree, 
                         tree_params,
                         cv = 5, 
                         n_jobs = -1,
                         verbose=True )

In [63]:
bad_dict = ['porno', 'porn', 'порно', 'порн','трах','sex','сиськи','boobs']
tmp = []
for x in df_train['clean_title']:
    flag = 0
    for bad_word in bad_dict:
        if bad_word in x:
            flag = 1
            
    tmp.append(flag)
df_train['bad_title'] = tmp

In [64]:
df_train.bad_title.value_counts()

0    124001
1     11308
Name: bad_title, dtype: int64

In [65]:
X_train = df_train[ [ 'dirt_url','bad_title', 'NB_title', 'LogReg_title'] ]

In [66]:
%%time
tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    5.6s


CPU times: user 1.75 s, sys: 295 ms, total: 2.05 s
Wall time: 9.11 s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    9.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=7,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 15, 3), 'max_features': range(1, 5), 'min_samples_leaf': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [67]:
f1_score(y_train, tree_grid.predict(X_train))

0.9982025164769323

# Submit

In [68]:
bad_dict = ['porno', 'porn', 'порно', 'порн','трах','sex','сиськи','boobs']
tmp = []
for x in df_test['clean_title']:
    flag = 0
    for bad_word in bad_dict:
        if bad_word in x:
            flag = 1
            
    tmp.append(flag)
df_test['bad_title'] = tmp

In [69]:
vect_title_test = tfidf_vectorizer.transform(df_test['clean_title'])

In [70]:
df_test['NB_title'] = MNB_model.predict(vect_title_test)

In [71]:
df_test['LogReg_title'] = LogReg_model.predict(vect_title_test)

In [72]:
df_test.head(100)

,id,url,title,in_black_list,title_tokens,clean_title,bad_title,NB_title,LogReg_title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...,0,кассацион суд самар начнет работ разн здан фот...,кассацион суд самар начнет работ разн здан фот...,0,0,0
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ...",0,индексац алимент как случа производ каков поря...,индексац алимент как случа производ каков поря...,0,0,0
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12,0,женщин импер мех part,женщин импер мех part,0,0,0
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ...",0,небрит волосат киск порн стран национальн онлайн,небрит волосат киск порн стран национальн онлайн,1,1,1
4,135313,2gis.ru,67,0,,,0,0,0
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп...",0,ваканс мерчендайзер визитн рязан групп компан ...,ваканс мерчендайзер визитн рязан групп компан ...,0,0,0
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...,0,смартфон appl iphon xr gb черн e куп интернет ...,смартфон appl iphon xr gb черн e куп интернет ...,0,0,0
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...,0,духов шкаф siem hb gys r sim dealer цен москв,духов шкаф siem hb gys r sim dealer цен москв,0,0,0
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...,0,ваканс аналитик департамент внедрен информацио...,ваканс аналитик департамент внедрен информацио...,0,0,0
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент,0,anoma defend v торрент,anoma defend v торрент,0,0,0


In [73]:
df_test['target'] = tree_grid.predict(
                            df_test[['dirt_url','bad_title', 'NB_title', 'LogReg_title']].values
                            ).astype(bool)

In [74]:
df_test[["id", "target"]].to_csv("ml_baselain15.csv", index=False)

!cat ml_baselain15.csv | head

id,target
135309,False
135310,False
135311,False
135312,True
135313,False
135314,False
135315,False
135316,False
135317,False
cat: stdout: Broken pipe
